<a href="https://colab.research.google.com/github/Sampath-Varma/Sampath_INFO5731_Fall2024/blob/main/Byrraju_Sampath_Assignment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Tuesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [1]:
!pip install requests beautifulsoup4 pandas

In [27]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_reviews(imdb_id, num_reviews=1000):
    reviews = []
    page = 0
    base_url = f"https://www.imdb.com/title/{imdb_id}/reviews?ref_=tt_ql_3"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    while len(reviews) < num_reviews:
        page += 1
        print(f"Scraping page {page} of reviews:")

        response = requests.get(base_url, headers=headers)
        if response.status_code == 403:
            print("Access denied (403).")
            break
        elif response.status_code != 200:
            print(f"Failed to retrieve page: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')

        review_containers = soup.find_all('div', class_='text show-more__control')

        if not review_containers:
            print("No more reviews found")
            break

        for container in review_containers:
            if len(reviews) >= num_reviews:
                break
            reviews.append(container.get_text(strip=True))

        print(f"Collected {len(reviews)} reviews so far")

        load_more_button = soup.find('div', class_='load-more-data')
        if load_more_button and len(reviews) < num_reviews:
            next_page_key = load_more_button.get('data-key')
            base_url = f"https://www.imdb.com/title/{imdb_id}/reviews/_ajax?ref_=undefined&paginationKey={next_page_key}"
        else:
            break

        time.sleep(1)

    return reviews[:num_reviews]

def save_reviews_to_csv(movie_title, movie_reviews, filename='imdb_reviews.csv'):
    df = pd.DataFrame(movie_reviews, columns=['Review'])
    df['Movie Title'] = movie_title
    df = df[['Movie Title', 'Review']]
    df.to_csv(filename, index=False)
    print(f"Saved {len(df)} reviews to {filename}")

def collect_movie_reviews():

    movie = {"title": "Dune: Part Two", "id": "tt15239678"}

    print(f"Scraping reviews for {movie['title']}:")
    reviews = scrape_reviews(movie['id'])

    if len(reviews) > 0:
        save_reviews_to_csv(movie['title'], reviews)
    else:
        print("No reviews collected")

if __name__ == "__main__":
    collect_movie_reviews()

Scraping reviews for Dune: Part Two:
Scraping page 1 of reviews:
Collected 25 reviews so far
Scraping page 2 of reviews:
Collected 50 reviews so far
Scraping page 3 of reviews:
Collected 75 reviews so far
Scraping page 4 of reviews:
Collected 100 reviews so far
Scraping page 5 of reviews:
Collected 125 reviews so far
Scraping page 6 of reviews:
Collected 150 reviews so far
Scraping page 7 of reviews:
Collected 175 reviews so far
Scraping page 8 of reviews:
Collected 200 reviews so far
Scraping page 9 of reviews:
Collected 225 reviews so far
Scraping page 10 of reviews:
Collected 250 reviews so far
Scraping page 11 of reviews:
Collected 275 reviews so far
Scraping page 12 of reviews:
Collected 300 reviews so far
Scraping page 13 of reviews:
Collected 325 reviews so far
Scraping page 14 of reviews:
Collected 350 reviews so far
Scraping page 15 of reviews:
Collected 375 reviews so far
Scraping page 16 of reviews:
Collected 400 reviews so far
Scraping page 17 of reviews:
Collected 425 revi

# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [28]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

file_path = '/content/imdb_reviews.csv'
df = pd.read_csv(file_path)
print("Initial Dataset:")
print(df.head(), "\n")

#1. Remove noise such as special characters and punctuations
def remove_noise(text):
    text = re.sub(r'[^A-Za-z\s]', '', text)
    return text
df['cleaned_review'] = df['Review'].apply(remove_noise)
print("After Removing Noise:")
print(df[['Review', 'cleaned_review']].head(), "\n")

#2. Remove numbers
def remove_numbers(text):
    text = re.sub(r'\d+', '', text)
    return text
df['cleaned_review'] = df['cleaned_review'].apply(remove_numbers)
print("After Removing Numbers:")
print(df[['Review', 'cleaned_review']].head(), "\n")

#3. Remove stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    return ' '.join(word for word in text.split() if word.lower() not in stop_words)
df['cleaned_review'] = df['cleaned_review'].apply(remove_stopwords)
print("After Removing Stopwords:")
print(df[['Review', 'cleaned_review']].head(), "\n")

#4. Lowercase all texts
df['cleaned_review'] = df['cleaned_review'].apply(lambda text: text.lower())
print("After Lowercasing:")
print(df[['Review', 'cleaned_review']].head(), "\n")

#5. Stemming
stemmer = PorterStemmer()
def stemming(text):
    return ' '.join(stemmer.stem(word) for word in text.split())
df['cleaned_review'] = df['cleaned_review'].apply(stemming)
print("After Stemming:")
print(df[['Review', 'cleaned_review']].head(), "\n")

#6. Lemmatization
lemmatizer = WordNetLemmatizer()
def lemmatization(text):
    return ' '.join(lemmatizer.lemmatize(word) for word in text.split())
df['cleaned_review'] = df['cleaned_review'].apply(lemmatization)
print("After Lemmatization:")
print(df[['Review', 'cleaned_review']].head(), "\n")

output_file_path = 'imdb_reviews_cleaned.csv'
df.to_csv(output_file_path, index=False)
print(f"Cleaned data saved to {output_file_path}")

Initial Dataset:
      Movie Title                                             Review
0  Dune: Part Two  This is what Hollywood needs. A great story wi...
1  Dune: Part Two  I'm going to write this as a review for both D...
2  Dune: Part Two  Had the pleasure to watch this film in an earl...
3  Dune: Part Two  Phenomenal stuff. I'll probably calm down tomo...
4  Dune: Part Two  If you liked or loved the first one, the same ... 

After Removing Noise:
                                              Review  \
0  This is what Hollywood needs. A great story wi...   
1  I'm going to write this as a review for both D...   
2  Had the pleasure to watch this film in an earl...   
3  Phenomenal stuff. I'll probably calm down tomo...   
4  If you liked or loved the first one, the same ...   

                                      cleaned_review  
0  This is what Hollywood needs A great story wit...  
1  Im going to write this as a review for both Du...  
2  Had the pleasure to watch this film in a

# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [30]:
# Your code here
!pip install nltk spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 82.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [31]:
import pandas as pd
import nltk
import spacy
from nltk import pos_tag, word_tokenize
from collections import Counter
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

file_path = '/content/imdb_reviews_cleaned.csv'
df = pd.read_csv(file_path)
nlp = spacy.load('en_core_web_sm')

#(1) Parts of Speech (POS) Tagging
def pos_analysis(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    return pos_tags
df['pos_tags'] = df['cleaned_review'].apply(pos_analysis)
pos_counts = Counter(tag for tags in df['pos_tags'] for _, tag in tags)
print("Parts of Speech Counts:")
print(f"Nouns: {pos_counts['NN'] + pos_counts['NNS'] + pos_counts['NNP'] + pos_counts['NNPS']}")
print(f"Verbs: {pos_counts['VB'] + pos_counts['VBD'] + pos_counts['VBG'] + pos_counts['VBN'] + pos_counts['VBP'] + pos_counts['VBZ']}")
print(f"Adjectives: {pos_counts['JJ'] + pos_counts['JJR'] + pos_counts['JJS']}")
print(f"Adverbs: {pos_counts['RB'] + pos_counts['RBR'] + pos_counts['RBS']}\n")

#(2) Dependency Parsing
def parse_sentences(text):
    doc = nlp(text)
    for sent in doc.sents:
        print(f"Sentence: {sent.text}")
        print("Dependency Parse:")
        for token in sent:
            print(f"{token.text:10} > {token.dep_:15} ({token.head.text})")
        print("\n")
example_sentence = df['cleaned_review'].iloc[0]
print("Example Analysis for the First Review Sentence:")
parse_sentences(example_sentence)

#(3) Named Entity Recognition (NER)
def ner_analysis(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities
df['entities'] = df['cleaned_review'].apply(ner_analysis)
entity_counts = Counter(ent[1] for entities in df['entities'] for ent in entities)
print("Named Entity Recognition Counts:")
print(entity_counts)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Parts of Speech Counts:
Nouns: 63338
Verbs: 14669
Adjectives: 23244
Adverbs: 5396

Example Analysis for the First Review Sentence:
Sentence: hollywood need great stori great directorproduc best thing studio get hell way let artist creat artdun part creativ beauti tragic mesmer never get bore anticip go happen next havent read book noth compar todeni villeneuv continu amaz effort put film act film top notch saw imax sound earth shatter your gonna see movi see largest screen possibl
Dependency Parse:
hollywood  > nsubj           (need)
need       > ROOT            (need)
great      > amod            (stori)
stori      > nmod            (studio)
great      > amod            (studio)
directorproduc > npadvmod        (best)
best       > amod            (thing)
thing      > compound        (studio)
studio     > nsubj           (get)
get        > ccomp           (need)
hell       > compound        (way)
way        > npadvmod        (let)
let        > advcl           (need)
artist     > compou

#**Comment**
Make sure to submit the cleaned data CSV in the comment section - 10 points

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [32]:
# Write your response below
'''
In the first question i have choosen the 2nd task which is to collect the top 1000 user reviews of Dune Part : 2 from IMDB which involved performing scrapping reviews where i have
used beautifulsoup and requets with appropriate headers to mimic a browser. I have implemented paginatio  to collect multiple pages of reviews untill i have reached 1000 reviews
and saved the 1000 reviews into a csv file. In the second question using the obtained dataset i had removed noise by eliminating special characters and punctuation form the text,
removed numbers, lowercased the text and also removed common words and applied stemming and lemmatization. These results were saved in a new csv file. Finally in the third question
I have conducted a syntax and structure analysis on the cleaned dataset which inlcudes POS tagging, parsing and NER. The main challenge i encountered was in the 1st question where I
couldn't get to scrape data due to access restrictions and even after that there was diffuculty of not getting all the 1000 reviews but i really enjoyed this process of scrapping the
reviews. I think the provided time for this assignment is sufficient to complete it.

'''

"\nIn the first question i have choosen the 2nd task which is to collect the top 1000 user reviews of Dune Part : 2 from IMDB which involved performing scrapping reviews where i have \nused beautifulsoup and requets with appropriate headers to mimic a browser. I have implemented paginatio  to collect multiple pages of reviews untill i have reached 1000 reviews \nand saved the 1000 reviews into a csv file. In the second question using the obtained dataset i had removed noise by eliminating special characters and punctuation form the text, \nremoved numbers, lowercased the text and also removed common words and applied stemming and lemmatization. These results were saved in a new csv file. Finally in the third question\nI have conducted a syntax and structure analysis on the cleaned dataset which inlcudes POS tagging, parsing and NER. The main challenge i encountered was in the 1st question where I\ncouldn't get to scrape data due to access restrictions and even after that there was diff